# Frequent Itemset Mining: Apriori Alternatives

In Module 3, we explored **apriori** method for mining frequent itemsets and inferring association rules. The key advantage of applying the apriori principle is that this method prunes a good amount of infrequent itemsets and rules. It is the first association rule mining algorithm that uses support-based pruning to systematically control the exponential growth of candidate itemsets.

**Apriori principle:** If an itemset is frequent, then all of its subsets must also be frequent. In other words, if an itemset is infrequent, then its supersets cannot be frequent. 

Despite its significant performance improvement, the apriori algorithm still incurs considerable I/O overhead since it requires making several passes over the transaction data set. E.g., let's think that the transaction database is a PostgreSQL database, and scanning a database is a costly operation. Several alternative methods have been developed to overcome these limitations and improve the efficiency of the apriori algorithm. In this notebook, we will explore **FP-Growth**. Also, we will explore **maximal frequent itemsets**, a compact representation for frequent itemsets. 

## Frequent Pattern Growth (FP-Growth)

This section presents an alternative algorithm called FP-growth that takes a different approach to indetifying frequent itemsets. Like the apriori method, this algorithm does not generate and test frequent itemsets. Instead, it encodes the dataset using a compact data structure called a frequent pattern tree (FP-tree) and extracts frequent itemsets directly from this structure. 

<img src="../images/fp-growth.png" height=400 width=600 />

*Fig. Frequent pattern growth method. The tree express the entire dataset as a compact representation. Here null is the root. We can traverse this tree from the root and identify frequent itemsets.*

The above fig shows a dataset that contains ten transactions and five items. Initially, the FP-tree contains only the root node represented by the null symbol. This fig shows the state of the tree after reading the first three and last transactions. Each node in the tree contains the label of an item and its count. The FP-tree is created by scanning the database twice. 

1. The data set is scanned once to determine the support count of each item. Infrequent items are discarded, while the frequent items are sorted in decreasing support counts 
2. Every transaction of the dataset is mapped onto one of the paths given in the FP-tree.

For this module we will be using [mlxtend](https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/) python package to demonstrate frequent pattern mining and association rule mining. In this notebook we explore a copmputer sales data, a toy one. 

 ## Import required Libraries

In [ ]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth, fpmax
import matplotlib.pyplot as plt
from fim import eclat
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline

## Data Loading

The data is located here: `/dsa/data/DSA-8410/association-mining/computer-sales.csv`


In [ ]:
df = pd.read_csv('/dsa/data/DSA-8410/association-mining/computer-sales.csv') 
df.head()

In [ ]:
print(f"Num of transactions = {df.shape[0]}")
print(f"Maximum num of items per transaction = {df.shape[1]}")

## Preprocessing 

To apply `apriori`, `fpgrowth`, and other frequent pattern methods, we need to convert the dataset to a binary incidence matrix. We can use pandas `get_dummies` method for this conversion.

In [ ]:
df = pd.get_dummies(df)

In [ ]:
df.head()

## A. Frequent Patterns with Apriori Method

Apriori module from mlxtend library provides fast and efficient apriori implementation.

```
Default Parameters: apriori(df, min_support=0.5, use_colnames=False, max_len=None, verbose=0, low_memory=False)"
```


In [ ]:
freq_items = apriori(df, min_support=0.2, use_colnames=True, verbose=1)
freq_items.shape

In [ ]:
freq_items.head()

For readability, let's swap the two columns. 

In [ ]:
freq_items = freq_items.reindex(columns=['itemsets', 'support'])
freq_items['length'] = freq_items['itemsets'].apply(lambda x: len(x))

In [ ]:
print(f"Total number of frequent itemsets = {freq_items.shape[0]}")
freq_items.tail()

### Rules from Frequent Itemsets

Once the frequent itemsets from transactions in a database have been found, it is straightforward to generate strong association rules from them (where strong association rules satisfy both minimum support and minimum confidence). This can be done using  confidence, which we show again here for completeness:

$\text{conf}(X \Rightarrow Y ) = P(Y|X) = \frac{\text{sup}(X \cup Y )}{\text{sup}(X)} = \frac{\text{count}(X \cup Y )}{\text{count}(X)}$. 


We now apply `association_rules` on the frequent itemsets found in the earlier step. The function definition is as follows: 

```
association_rules(df, metric=’confidence’, min_threshold=0.8, support_only=False)
```

Metric can be set to `confidence`, `lift`, `support`, `leverage` and `conviction`.

In [ ]:
rules = association_rules(freq_items, metric="confidence", min_threshold=0.75)
print(f"Total number of rules = {rules.shape[0]}")
rules.head()

Based on the given measures such as supports, confidence, lift, leverage, and conviction, we can subset rules and inspect them. E.g, let's see the rules that has occured at least 20% of the time with at least 80% of confidence. 


In [ ]:
rules[(rules['support'] >= .20) & (rules['confidence'] >= .80)]

We can see all but one rules have the highest confidence and coviction. We should not take this result seriously as there are only 28 transactions. Let's explore some surprising patterns with lift and leverage. 

In [ ]:
plt.scatter(rules['lift'], rules['leverage'], alpha=0.5)
plt.xlabel('lift')
plt.ylabel('leverage')
plt.title('Lift vs Leverage');
plt.show()

In [ ]:
rules[(rules['lift'] >= 2) & (rules['leverage'] >= .10)]

Now, we will explore the FP-Growth method, which does not employ candidate generation and testing strategy. Instead, this method infers a compact tree (aka FP tree) representation of the entire dataset and provides rules by traversing this dataset. This method usually runs much faster than the apriori method. 

## B. Frequent Patterns with FP-Growth



In [ ]:
freq_items_fp = fpgrowth(df, min_support=0.2, use_colnames=True)

# swap columns for readability 
freq_items_fp = freq_items_fp.reindex(columns=['itemsets', 'support'])
freq_items_fp['length'] = freq_items_fp['itemsets'].apply(lambda x: len(x))

In [ ]:
print(f"Total number of frequent itemsets = {freq_items_fp.shape[0]}")
freq_items_fp.tail()

As expected, we get the 42 frequent patterns like the apriori method. Now we can pass these frequent patterns to `association_rules` method for inferring rules. 

In [ ]:
rules_fp = association_rules(freq_items_fp, metric="confidence", min_threshold=0.75)
print(f"Total number of rules = {rules_fp.shape[0]}")
rules_fp.head()

## Apriori vs FP-Growth 

Why use FP-Growth. As FP-Growth doesn't require creating candidate sets explicitly, it can be magnitudes faster than the alternative Apriori algorithm. For instance, the following cells compare the performance of the Apriori algorithm to the performance of FP-Growth -- even in this very simple retail dataset scenario, FP-Growth is about 2 times faster.

In [ ]:
## Time taken by apriori: n: inner loop; r: outer loop
%timeit -n 100 -r 10 apriori(df, min_support=0.2)



In [ ]:
## Time taken by FP-Growth
%timeit -n 100 -r 10 fpgrowth(df, min_support=0.2)

## C. Compact Representation of Frequent Itemsets

In a real-world scenario, the number of frequent itemsets produced could be very large. It is useful to identify a small representative set of itemsets from which all other frequent itemsets can be derived. Two such representations are **maximal** and **closed** frequent itemsets. 

### Maximal Frequent Itemsets

A **maximal frequent itemset** is defined as a frequent itemset for which none of its immediate supersets are frequent. Maximal frequent itemsets effectively provide a compact representation of frequent itemsets. We can derive all frequent itemsets from a maximal frequent itemsets. 



<img src="../images/maximal-itemset.png" height=400, width=600 />

**Fig. Maximal frequent itemset.** _This figure shows the lattice of itemsets for five items {a, b, c, d, e}. In this toy example the boundary between frequent and infrequent itemsets are shown. An itemset such as {a,d} is maximal frequent because all of its immediate supersets, {a,b,d}, {a,c,d}, and {a,d,e}, are infrequent. In contrast, {a,c} is non-maximal becauseone of its immediate supersets, {a,c,e}, is frequent. (source: Intro to Data Mining byTan et al.)_

The sizeof an FP-tree is typically smaller than the size of the uncompressed data because many transactions in market basket data often share a few items in common.

### Closed Frequent Itemsets

An **closed itemset** is the one for which none of its immediate supersets has the same support count as the itemset. In other words, an itemsetset X is not closed if at least one of its immediate supersets has the same support count as X. 

An itemset is a **closed frequent itemset** if it is closed and its support is greater than or equal to user-defined minimum support.

<img src="../images/closed-itemset.png" height=400 width = 600 />

**Fig. Example of closed frequent itemsets.** _As the node { b, c }  is associated with transaction ids 1, 2, and 3, its support count is equal to three. From the transactions given in this diagram, notice that every transaction that contains b also contains c. Consequently,the support for {b} is identical to {b, c} and {b} should not be considered a closed itemset. On the other hand, {b, c} is a closed itemset because it does not have the same support count as any of its supersets._

**Maximal frequent itemsets are subset of closed frequent itemsets.** Now, we use `fpmax` method to identify all maximal frequent patterns with min support 0.2 for this dataset.

In [ ]:
max_patterns = fpmax(df, min_support=0.2, use_colnames=True)

In [ ]:
# for readability 
max_patterns = max_patterns.reindex(columns=['itemsets', 'support'])
max_patterns['length'] = max_patterns['itemsets'].apply(lambda x: len(x))

In [ ]:
print(f"Total number of maximal frequent patterns = {max_patterns.shape[0]}")
max_patterns

There are 20 maximal frequent itemsets which are almost half of the total number (42) of frequent patterns. 

In [ ]:
%timeit -n 100 -r 10 fpmax(df, min_support=0.2)

The method runs as fast as the FP-Growth method. 

# Save your notebook, then `File > Close and Halt`